<h1>Cascade Error Correction Protocol</h1>
<hr>
<i>Written by Jamie</i><br>
<p style="margin-bottom:1.5cm;">This script is to test how to implement the error correction protocol <a href="https://link.springer.com/chapter/10.1007/3-540-48285-7_35">cascade</a> into the quantum key distribution visualisation and basic simulation script "<i>quantum-key-distro.ipynb</i>".</p>

<h3>Import packages</h3>
<hr>
<p>Import python packages: <a href="https://pypi.org/project/random2/">random</a> and the <a href="https://pypi.org/project/tabulate/">tabulate</a> module from the library that shares its name.

In [469]:
import random
from tabulate import tabulate

<h3>Properties of Transaction</h3>
<hr> 
<p>
    Prompt the user for numerical inputs for:
    <ul>
        <li>The length in bits of the random string being generated and sent by the sender "Alice".</li>
        <li>Probability of an incorrect bit being received by the receiver "Bob" due to noise in the system, also known as the error rate.</li>
    </ul>
    And run input validation on these user inputs to ensure that they are in the correct format so the program doesn't break later on. If the input validation fails, ask the user for the input again, providing the reason why the input was not sufficient.
</p>

In [470]:
## Number of random bits
no_bits=input("Number of random bits: ")
valid=False
while valid == False: # Input validation
    try:
        no_bits=int(no_bits)
    except:
        no_bits=input("-----\nError: Value must be a number\nNumber of random bits: ")
    else:
        if no_bits <= 0:
            no_bits=input("-----\nError: Value must be above 0\nNumber of random bits: ")
        else:
            valid=True
print ("Number of random bits: "+str(no_bits)) # Print value for debugging or later reference

## Probability of an incorrect bit being received due to noise (error rate)
error_rate=input("Probability of an incorrect bit being received due to noise (error rate) [%]: ")
valid=False
while valid == False: # Input validation
    try:
        error_rate=int(error_rate)
    except:
        error_rate=input("-----\nError: Value must be a number\nProbability of an incorrect bit being received due to noise (error rate) [%]: ")
    else:
        if error_rate > 100 or error_rate < 0:
            error_rate=input("-----\nError: Value must be between 0 and 100\nProbability of an incorrect bit being received due to noise (error rate) [%]: ")
        else:
            valid=True
print ("Probability of an incorrect bit being received due to noise (error rate): "+str(error_rate)+"%") # Print value for debugging or later reference

Number of random bits: 16
Probability of an incorrect bit being received due to noise (error rate): 50%


<h3>Define functions</h3>
<hr>
<p>To reduce code sprawl and to make life easier for programming, a number of functions are defined that can be recalled by the program later.<br>
These include:
<ul>
    <li>A function to convert strings to arrays which is used to convert the randomly generated strings of 0s and 1s or Ds and Rs into arrays that are easy to preform calculations on.</li>
    <li>A function to convert arrays to strings which is helpful in debugging/user output to print out easier-to-read strings of values in arrays, without the square brackets and commas.</li>
    <li>A function to generate a random array of 1s and 0s that is as long as the integer inputted into the function (used for making Alice's random sending bits).</li>
    <li>A function to generate a random array of Ds and Rs that is as long as the integer inputted into the function (used for making Alice and Bob's encoding and decoding bases).</li>
    <li>A function that returns true or false with a probability of the integer that is inputted into it (used for making a percentage of the photons get lost in transmission and for making a percentage of the received bits contain errors).</li>
</ul>
</p>

In [471]:
def string_to_array(string): # Used for user display purposes to convert each character in a string to entries in an array
    array=[]
    for i in string: # iterate through each character in the string
        array.append(i) # append each character to the array
    return array

def array_to_string(array): # Used for user display purposes to convert each entry in an array to characters in a string
    string=""
    for i in array: # iterate through each entry in the array
        string = string + i # append each entry to the string
    return string

def generate_random_bits(length): # Used to generate a random array of 1s and 0s of a given length
    binary_string = ''.join(random.choice('01') for _ in range(length)) # generate a random string of 1s and 0s of a given length
    return string_to_array(binary_string) # Convert the string to an array and return it

def generate_random_bases(length): # Used to generate a random array of Ds and Rs of a given length
    bases_string = ''.join(random.choice('DR') for _ in range(length)) # generate a random string of Ds and Rs of a given length
    return string_to_array(bases_string) # Convert the string to an array and return it

def test_percentage(prob): # Essentially a coin flip with a given probability, has the given probability of returning true 
    # generate a random float between 0 and 1
    if random.random() < (prob/100): # if the float is less than the probability as a decimal, return true
        return True
    else: # otherwise, return false
        return False
    
def parity_sum(array): # Used to calculate the sum of all entries in an array
    total=0
    for i in array: # iterate through each entry in the array
        i = int(i)
        total=total+i # add each entry to the total
        if total == 2: # binary addition, so if the total is 2, set it to 0
            total=0
    return total

def split_array(array1, array2): # Used to split two arrays into two halves
    low_array1=[]
    up_array1=[]
    low_array2=[]
    up_array2=[]
    LB=0 # set the lower bound to 0
    UB=len(array1)-1 # set the upper bound to the length of the array minus 1
    mid=(LB+UB)//2 # find the middle index
    for i in range(0,len(array1)):
        if i <= mid:
            low_array1.append(array1[i])
            low_array2.append(array2[i])
        else:
            up_array1.append(array1[i])
            up_array2.append(array2[i])
    if len(up_array1) != len(up_array2) or len(low_array1) != len(low_array2) or parity_sum(low_array1) == parity_sum(low_array2) and parity_sum(up_array1) == parity_sum(up_array2):
        return "split_array() Error: The lengths of the arrays are not equal or the parity sums of the arrays are equal"
    elif parity_sum(low_array1) != parity_sum(low_array2) and parity_sum(up_array1) == parity_sum(up_array2):
        return [low_array1, low_array2, "low"]
    elif parity_sum(low_array1) == parity_sum(low_array2) and parity_sum(up_array1) != parity_sum(up_array2):
        return [up_array1, up_array2, "up"]
    else:
        return "Error"


bounds=[]
def cascade(correct_array, incorrect_array):
    global split_arrays
    split_arrays=[]
    global split_arrays2
    split_arrays2=[]
    global bounds
    bounds=[]
    while len(correct_array) != 1:
        both_arrays = split_array(correct_array, incorrect_array)
        correct_array = both_arrays[0]
        incorrect_array = both_arrays[1]
        if both_arrays[2] == "low":
            bounds.append("low")
        elif both_arrays[2] == "up":
            bounds.append("up")
        else:
            return "Error"
        split_arrays.append(produce_neat_array(incorrect_array, bounds, "Bob"))
        split_arrays2.append(produce_neat_array(correct_array, bounds, "Alice"))
    return(incorrect_array)

def produce_neat_array(array, bounds, person):
    if person == "Alice":
        neat_array = ["Alice"]
    if person == "Bob":
        neat_array = ["Bob"]
    length = no_bits
    for i in bounds:
        if i == "up":
            length = length - (length//2)
            for j in range(0,length):
                neat_array.append("")
        elif i == "low":
            length = length - (length//2)
    for i in array:
        neat_array.append(i)
    for i in range(0,no_bits-len(neat_array)):
        neat_array.append("")
    neat_array.append("")
    neat_array.append("")
    return neat_array

<h3>Generate Bits With Errors</h3>
<hr>
<p>
    <ul>
        <li>Generate a random string of bits (1s and 0s) for Alice.</li>
        <li>Copy Alice's bits to Bob's and apply the error rate probability to each bit to produce a string with errors</li>
    </ul>
</p>

In [472]:
## Alice's bits
alice_bits=generate_random_bits(no_bits) # Generate Alice's random bits

## Bob's bits
bob_bits=[] # Initialise Bob's bits array
error_array=[] # Initialise error array for user display purposes
no_errors=0 
for bit in alice_bits: # iterate through each bit in Alice's bits
    if test_percentage(error_rate) == True: # if an error occurs in transmission, flip the bit
            if bit == "1":
                bob_bits.append("0")
            elif bit == "0":
                bob_bits.append("1")
            else:
                print("Error: alice bit is not 1 or 0") # Error message for debugging
            error_array.append("X")
            no_errors+=1
    else: # otherwise, append the bit to Bob's bits array
        bob_bits.append(bit)
        error_array.append("")

<h3>The Cascade Protocol Itself</h3>
<hr>
<p>This is where the cascade protocol is started, the parity sum of both Alice and Bob's bits are calculated and if they are different, a binary search for the error bit is started.

In [473]:
alice_parity, bob_parity = parity_sum(alice_bits), parity_sum(bob_bits) # Calculate the parity of Alice's and Bob's bits
if alice_parity == bob_parity: # If the parity of Alice's and Bob's bits are the same, do not use the cascade method
    print("Parity of Alice's bits: "+str(alice_parity)+"\nParity of Bob's bits: "+str(bob_parity)+"\nParity is the same, no cascade method required")
else:
    wrong_bit=cascade(alice_bits,bob_bits) # Otherwise, use the cascade method to find the error

<h3>Display Each Stage in a Table</h3>
<hr>
<p>A label is inserted at the start of each array so that it is labelled in the table then each stage of the Cascade Protocol (represented by the data from that stage in an array) is combined into a 2D array and then displayed as a table for easy viewing of how the protocol produced corrected errors in a stage-by-stage manner.</p>

In [474]:
print("\033[1m"+ "\033[4m" + "Error Correction - Cascade Protocol" + "\033[0m" + "\033[0m") # bold and underline table title
# Add label to each array for tabulation
alice_bits.insert(0,'Alice\'s bits')
alice_bits.append('Parity Value:')
alice_bits.append(alice_parity)
bob_bits.insert(0,'Bob\'s bits')
bob_bits.append('Parity Value:')
bob_bits.append(bob_parity)
error_array.insert(0, 'Error?')
error_array.append('Total Errors:')
error_array.append(no_errors)
data = [alice_bits,bob_bits,error_array]
for i in range(0,len(split_arrays)):
    data.append(split_arrays2[i])
    data.append(split_arrays[i])
print(tabulate(data, tablefmt="simple_grid")) # Print table
# Remove labels from each array so they can be used for further calculations
for array in data:
    if array == alice_bits or array == bob_bits or array == error_array:
        array.pop(0)
        array.pop(-1)
        array.pop(-1)
print("Error rate: "+str(no_errors/no_bits*100)+"%") # Print error rate
print("Parity bits:", len(split_arrays)*2) # Print number of parity bits

Error Correction - Cascade Protocol
┌──────────────┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───────────────┬────┐
│ Alice's bits │ 0 │ 1 │ 1 │ 1 │ 1 │ 0 │ 0 │ 1 │ 1 │ 1 │ 0 │ 0 │ 0 │ 0 │ 0 │ 0 │ Parity Value: │  1 │
├──────────────┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───────────────┼────┤
│ Bob's bits   │ 1 │ 1 │ 0 │ 0 │ 0 │ 1 │ 1 │ 0 │ 0 │ 0 │ 1 │ 0 │ 1 │ 1 │ 0 │ 1 │ Parity Value: │  0 │
├──────────────┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───────────────┼────┤
│ Error?       │ X │   │ X │ X │ X │ X │ X │ X │ X │ X │ X │   │ X │ X │   │ X │ Total Errors: │ 13 │
├──────────────┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───────────────┼────┤
│ Alice        │ 0 │ 1 │ 1 │ 1 │ 1 │ 0 │ 0 │ 1 │   │   │   │   │   │   │   │   │               │    │
├──────────────┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───────────────┼────┤
│ Bob          │ 1 │ 1 │ 0 │ 0 │ 0 │ 1 │ 1 │ 0

<h3>Cascade Protocol</h3>
<hr>
<p>Cascade uses a very similar process to a binary search to split the strings in half and search for the error